In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [51]:
# input
pvar_f <- '/oak/stanford/groups/mrivas/ukbb24983/exome/pgen/oqfe_2020/ukb24983_exomeOQFE.pvar.zst'
data_d <- '/oak/stanford/groups/mrivas/ukbb24983/exome/annotation/20201025_exome_oqfe_2020'
afreq_hwe_out_d <- file.path(str_replace(data_d, '/oak/stanford/', '/scratch/'), 'afreq_hwe')
# output
out_f <- file.path(data_d, 'ukb24983_exomeOQFE.afreq_hwe.20201025.pvar')

# constants
pops <- c('white_british', 'non_british_white', 'african', 's_asian', 'e_asian', 'related', 'others')


In [20]:
# functions

cat_or_zcat <- function(f){
    ifelse(endsWith(f, '.zst'), 'zstdcat', ifelse(endsWith(f, '.gz'), 'zcat', 'cat'))
}

fread_CHROM <- function(f, select=NULL){
    fread(cmd=paste(cat_or_zcat(f), f), colClasses = c('#CHROM'='character'), select=select) %>% rename('CHROM'='#CHROM')
}

read_tbl <- function(pops, prefix, suffix, select=NULL){
    pops %>% 
    lapply(function(pop){
        ifelse(pop == 'all', sprintf('%s.%s', prefix, suffix), sprintf('%s.%s.%s', prefix, pop, suffix)) %>%
        fread_CHROM(select=select) %>% mutate(population = pop)
    }) %>% bind_rows()
}

spread_pop <- function(df, col, pops, col_prefix, col_suffix, col_sep='_'){    
    df %>% select(ID, col, population) %>%
    rename(!!'VAL' := col) %>% spread(all_of(population), VAL) %>%
    select(ID, all_of(pops)) %>% rename(setNames(
        1:length(pops)+1, 
        paste(rep(col_prefix, length(pops)), pops, rep(col_suffix, length(pops)), sep=col_sep)
    ))
}


## read files

### `pvar` file

In [5]:
pvar_f %>% fread_CHROM() -> pvar_df


### `afreq` file

In [23]:
read_tbl(
    c('all', pops), file.path(afreq_hwe_out_d, 'ukb24983_exomeOQFE'), 'afreq.zst', select=c('#CHROM', 'ID', 'ALT_FREQS')
) %>% select(-CHROM) -> afreq_long_df


In [25]:
dim(pvar_df) %>% print()
dim(afreq_long_df) %>% print()
print(8 * 17777950)


[1] 17777950        5
[1] 142223600         3
[1] 142223600


In [26]:
afreq_long_df %>%
spread_pop('ALT_FREQS', c('all', pops), 'UKB', 'AF') %>%
rename('UKB_AF'=2) -> afreq_wide_df


In [27]:
dim(afreq_wide_df) %>% print()


[1] 17777950        9


In [28]:
rm(afreq_long_df)


### `hwe` file

In [30]:
read_tbl(
    c('all', pops), file.path(afreq_hwe_out_d, 'ukb24983_exomeOQFE'), 'hardy.zst',
    select=c('#CHROM', 'ID', 'MIDP')
) %>% select(-CHROM) -> hwe_autosomes_long_df


In [31]:
read_tbl(
    c('all', pops), file.path(afreq_hwe_out_d, 'ukb24983_exomeOQFE'), 'hardy.x.zst',
    select=c('#CHROM', 'ID', 'MIDP')
) %>% select(-CHROM) -> hwe_chrX_long_df


In [32]:
bind_rows(hwe_autosomes_long_df, hwe_chrX_long_df) %>% 
rename('hwe_p'='MIDP') -> hwe_long_df


In [33]:
hwe_autosomes_long_df %>% dim() %>% print()
hwe_chrX_long_df      %>% dim() %>% print()
hwe_long_df           %>% dim() %>% print()


[1] 139374240         3
[1] 2848288       3
[1] 142222528         3


In [34]:
139374240 + 2848288


[1] 142222528

In [35]:
rm(hwe_autosomes_long_df)
rm(hwe_chrX_long_df)

In [36]:
hwe_long_df %>%
spread_pop('hwe_p', c('all', pops), 'UKB', 'hwe_p')  %>%
rename('UKB_hwe_p'=2) -> hwe_wide_df


In [37]:
hwe_wide_df           %>% dim() %>% print()


[1] 17777816        9


In [38]:
17777816 * 8 

[1] 142222528

In [39]:
rm(hwe_long_df)

### `gcount` file

In [11]:
read_tbl(
    c('all', pops), file.path(afreq_hwe_out_d, 'ukb24983_exomeOQFE'), 'gcount.zst',
    select=c('#CHROM', 'ID', 'OBS_CT', 'MISSING_CT', 'HOM_REF_CT', 'HET_REF_ALT_CTS', 'TWO_ALT_GENO_CTS', 'HAP_REF_CT', 'HAP_ALT_CTS')
) %>% select(-CHROM) -> gcount_long_df


In [40]:
dim(gcount_long_df)

[1] 142223600         9

## join data frames

### `AF` and `hwe_p`


In [41]:
pvar_df %>%
left_join(afreq_wide_df, by='ID') -> pvar_af_hwe_df


In [42]:
pvar_af_hwe_df %>%
left_join(hwe_wide_df, by='ID') -> pvar_af_hwe_df


In [43]:
pvar_af_hwe_df %>% dim() %>% print()

[1] 17777950       21


In [44]:
pvar_af_hwe_df %>% colnames()

[1] "CHROM"                       "POS"                        
 [3] "ID"                          "REF"                        
 [5] "ALT"                         "UKB_AF"                     
 [7] "UKB_white_british_AF"        "UKB_non_british_white_AF"   
 [9] "UKB_african_AF"              "UKB_s_asian_AF"             
[11] "UKB_e_asian_AF"              "UKB_related_AF"             
[13] "UKB_others_AF"               "UKB_hwe_p"                  
[15] "UKB_white_british_hwe_p"     "UKB_non_british_white_hwe_p"
[17] "UKB_african_hwe_p"           "UKB_s_asian_hwe_p"          
[19] "UKB_e_asian_hwe_p"           "UKB_related_hwe_p"          
[21] "UKB_others_hwe_p"

In [45]:
rm(pvar_df)
rm(afreq_wide_df)
rm(hwe_wide_df)


### Other outputs from `gcount` file

In [46]:
for(col in c('OBS_CT', 'MISSING_CT', 'HOM_REF_CT', 'HET_REF_ALT_CTS', 'TWO_ALT_GENO_CTS', 'HAP_REF_CT', 'HAP_ALT_CTS')){
    message(col)
    
    gcount_long_df %>%
    spread_pop(col, c('all', pops), 'UKB', col) %>%
    rename(!!sprintf('UKB_%s', col) := 2) -> tmp
    
    pvar_af_hwe_df %>% left_join(tmp, by='ID') -> pvar_af_hwe_df
    
    rm(tmp)
}
s

OBS_CT

MISSING_CT

HOM_REF_CT

HET_REF_ALT_CTS

TWO_ALT_GENO_CTS

HAP_REF_CT

HAP_ALT_CTS



## compute the missingness

In [48]:
pvar_af_hwe_df %>% 
mutate(
    f_miss = UKB_MISSING_CT / (UKB_MISSING_CT + UKB_OBS_CT)
) -> pvar_af_hwe_df


In [49]:
pvar_af_hwe_df %>% dim()

[1] 17777950       78

In [55]:
pvar_af_hwe_df %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)


In [52]:
out_f

[1] "/oak/stanford/groups/mrivas/ukbb24983/exome/annotation/20201025_exome_oqfe_2020/ukb24983_exomeOQFE.afreq_hwe.20201025.pvar"

In [53]:
pvar_af_hwe_df %>% colnames()

[1] "CHROM"                                 
 [2] "POS"                                   
 [3] "ID"                                    
 [4] "REF"                                   
 [5] "ALT"                                   
 [6] "UKB_AF"                                
 [7] "UKB_white_british_AF"                  
 [8] "UKB_non_british_white_AF"              
 [9] "UKB_african_AF"                        
[10] "UKB_s_asian_AF"                        
[11] "UKB_e_asian_AF"                        
[12] "UKB_related_AF"                        
[13] "UKB_others_AF"                         
[14] "UKB_hwe_p"                             
[15] "UKB_white_british_hwe_p"               
[16] "UKB_non_british_white_hwe_p"           
[17] "UKB_african_hwe_p"                     
[18] "UKB_s_asian_hwe_p"                     
[19] "UKB_e_asian_hwe_p"                     
[20] "UKB_related_hwe_p"                     
[21] "UKB_others_hwe_p"                      
[22] "UKB_OBS_CT"                            
[23] "UKB_white_british_OBS_CT"              
[24] "UKB_non_british_white_OBS_CT"          
[25] "UKB_african_OBS_CT"                    
[26] "UKB_s_asian_OBS_CT"                    
[27] "UKB_e_asian_OBS_CT"                    
[28] "UKB_related_OBS_CT"                    
[29] "UKB_others_OBS_CT"                     
[30] "UKB_MISSING_CT"                        
[31] "UKB_white_british_MISSING_CT"          
[32] "UKB_non_british_white_MISSING_CT"      
[33] "UKB_african_MISSING_CT"                
[34] "UKB_s_asian_MISSING_CT"                
[35] "UKB_e_asian_MISSING_CT"                
[36] "UKB_related_MISSING_CT"                
[37] "UKB_others_MISSING_CT"                 
[38] "UKB_HOM_REF_CT"                        
[39] "UKB_white_british_HOM_REF_CT"          
[40] "UKB_non_british_white_HOM_REF_CT"      
[41] "UKB_african_HOM_REF_CT"                
[42] "UKB_s_asian_HOM_REF_CT"                
[43] "UKB_e_asian_HOM_REF_CT"                
[44] "UKB_related_HOM_REF_CT"                
[45] "UKB_others_HOM_REF_CT"                 
[46] "UKB_HET_REF_ALT_CTS"                   
[47] "UKB_white_british_HET_REF_ALT_CTS"     
[48] "UKB_non_british_white_HET_REF_ALT_CTS" 
[49] "UKB_african_HET_REF_ALT_CTS"           
[50] "UKB_s_asian_HET_REF_ALT_CTS"           
[51] "UKB_e_asian_HET_REF_ALT_CTS"           
[52] "UKB_related_HET_REF_ALT_CTS"           
[53] "UKB_others_HET_REF_ALT_CTS"            
[54] "UKB_TWO_ALT_GENO_CTS"                  
[55] "UKB_white_british_TWO_ALT_GENO_CTS"    
[56] "UKB_non_british_white_TWO_ALT_GENO_CTS"
[57] "UKB_african_TWO_ALT_GENO_CTS"          
[58] "UKB_s_asian_TWO_ALT_GENO_CTS"          
[59] "UKB_e_asian_TWO_ALT_GENO_CTS"          
[60] "UKB_related_TWO_ALT_GENO_CTS"          
[61] "UKB_others_TWO_ALT_GENO_CTS"           
[62] "UKB_HAP_REF_CT"                        
[63] "UKB_white_british_HAP_REF_CT"          
[64] "UKB_non_british_white_HAP_REF_CT"      
[65] "UKB_african_HAP_REF_CT"                
[66] "UKB_s_asian_HAP_REF_CT"                
[67] "UKB_e_asian_HAP_REF_CT"                
[68] "UKB_related_HAP_REF_CT"                
[69] "UKB_others_HAP_REF_CT"                 
[70] "UKB_HAP_ALT_CTS"                       
[71] "UKB_white_british_HAP_ALT_CTS"         
[72] "UKB_non_british_white_HAP_ALT_CTS"     
[73] "UKB_african_HAP_ALT_CTS"               
[74] "UKB_s_asian_HAP_ALT_CTS"               
[75] "UKB_e_asian_HAP_ALT_CTS"               
[76] "UKB_related_HAP_ALT_CTS"               
[77] "UKB_others_HAP_ALT_CTS"                
[78] "f_miss"

In [54]:
str_replace(out_f, '.pvar$', '.compact.pvar')

[1] "/oak/stanford/groups/mrivas/ukbb24983/exome/annotation/20201025_exome_oqfe_2020/ukb24983_exomeOQFE.afreq_hwe.20201025.compact.pvar"

In [57]:
pvar_af_hwe_df %>%
select(CHROM, POS, ID, REF, ALT, f_miss, UKB_white_british_hwe_p, UKB_white_british_AF, UKB_AF) %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(str_replace(out_f, '.pvar$', '.compact.pvar'), sep='\t', na = "NA", quote=F)


In [ ]:
pvar_af_hwe_df %>% select(ID, UKB_white_british_hwe_p) %>% 
rename('MIDP' = 'UKB_white_british_hwe_p') %>% plot_cumsum() + 
labs(title='Exome 200k HWE p-value (white British)') -> p


In [71]:
plot_cumsum <- function(df){
    df %>% ggplot(aes(x = VALUE)) + 
    stat_bin(aes(y=cumsum(..count..)/nrow(df)),geom="line",color="black", bins=100) +
    labs(
        y = 'cumsum(n) / n '
    ) + theme_bw() + theme(
        legend.title = element_text(size=16),
        legend.text  = element_text(size=16),
        axis.text=element_text(size=16), 
        axis.title=element_text(size=16)
    ) +
    geom_hline(yintercept = 1, color='gray')
}


In [77]:
pvar_af_hwe_df %>% select(ID, UKB_white_british_hwe_p) %>% 
mutate(VALUE=-log10(UKB_white_british_hwe_p + .Machine$double.xmin)) %>%
plot_cumsum() + xlim(0, 20) + ylim(0, 1) + 
labs(x = latex2exp::TeX('HWE (midp) -log_{10}(P)'), title='Exome 200k HWE p-value (white British)') +
geom_vline(xintercept = 7, color='red') -> p_hwe


In [78]:
suppressWarnings(suppressPackageStartupMessages({
for(ext in c('png', 'pdf')){
    ggsave(
        sprintf('6_hwe_midp_plot.%s', ext),
        p_hwe, width=6,height=6
    )    
}
}))


In [75]:
pvar_af_hwe_df %>% select(ID, f_miss) %>% 
mutate(VALUE=-log10(f_miss + .Machine$double.xmin)) %>%
plot_cumsum() + xlim(0, 6) + ylim(0, 1) + 
labs(x = latex2exp::TeX('-log_{10}(Missigness)'), title='Exome 200k missigness (per variant)') +
geom_vline(xintercept = 3, color='red') -> p_miss


In [76]:
suppressWarnings(suppressPackageStartupMessages({
for(ext in c('png', 'pdf')){
    ggsave(
        sprintf('6_miss_plot.%s', ext),
        p_miss, width=6,height=6
    )    
}
}))


In [85]:
pvar_af_hwe_df %>% select(ID, UKB_white_british_AF) %>% 
mutate(VALUE=-log10(pmin(1-UKB_white_british_AF, UKB_white_british_AF) + .Machine$double.xmin)) %>%
plot_cumsum() + xlim(0, 6) + ylim(0, 1) + 
labs(x = latex2exp::TeX('-log_{10}(minor allele frequency)'), title='Exome 200k minor allele frequency in white British subset') +
geom_vline(xintercept = -log10(1/(2*137920)), color='red') -> p_maf_wb


In [87]:
suppressWarnings(suppressPackageStartupMessages({
for(ext in c('png', 'pdf')){
    ggsave(
        sprintf('6_maf_wb_plot.%s', ext),
        p_maf_wb, width=6,height=6
    )    
}
}))


In [86]:
-log10(1/(2*137920))

[1] 5.440657